# Mapping Planter & Harvester GPS coordinates

It is commonplace for agricultural data to contain spatial information typically the longitude and latitude at which each data point was collected. However, one of the challenges this introduces is that different datasets often do not contain data that were collected at the same locations. A specific example involves planter and harvester data from precision farming equipment. As a planter moves through a field, it records thousands of measurements of its location along with various data about the planter operation (such as the variety of seed planted, the seeding rate, and the spacing between seeds). Similarly, as a harvester moves through a field, it records thousands of measurements of its location and the crop’s yield at that point. A question of agricultural interest is to estimate the relationship between yield and the variables measured during planter. However, the harvester data points do not necessarily exactly overlap the planter data points, so before any analysis can be conducted to probe these relationships, it’s necessary to determine which planter points are associated with which harvesterpoints.

Let's devise an algorithm to efficiently associate the planter data points with the harvester data points for one corn field (from the same year).

## Load the libraries

In [1]:
suppressMessages({
    library(ggmap)
    library(geosphere)
    library(dplyr)
    library(data.table)
    library(ggplot2)
    library(cowplot)
    })
# Set numeric digit precision
options(digits=11)

## Load the data files

The files contain a row for each data point collected by the planter and harvester machines. The columns in each file are as follows:

**planter_data.csv**
1. **long**: longitude where data point was collected.
2. **lat**: latitude where data point was collected.
3. **variety**: the seed variety planted at that location.
4. **seeding_rate**: continuous variable specifying the number of seeds planted per acre (in thousands).
5. **seed_spacing**: continuous variable specifying the distance between seeds (inches).

**harvester_data.csv**
1. **long**: longitude where data point was collected
2. **lat**: latitude where data point was collected
3. **yield**: continuous variable specifying the yield of the crop (in bushels/acre).

In [2]:
harvester <- readr::read_csv('harvester_data.csv')
planter <- readr::read_csv('planter_data.csv')

Parsed with column specification:
cols(
  long = col_double(),
  lat = col_double(),
  yield = col_double()
)
Parsed with column specification:
cols(
  long = col_double(),
  lat = col_double(),
  variety = col_character(),
  seeding_rate = col_double(),
  seed_spacing = col_double(),
  speed = col_double()
)


In [3]:
glimpse(harvester)

Observations: 16,626
Variables: 3
$ long  <dbl> -91.849991029, -91.849991529, -91.849991821, -91.849992445, -...
$ lat   <dbl> 40.376097181, 40.376106399, 40.376116310, 40.376126812, 40.37...
$ yield <dbl> 88.451156135, 102.173517338, 191.420756250, 236.556216966, 22...


In [4]:
glimpse(planter)

Observations: 6,314
Variables: 6
$ long         <dbl> -91.8505320, -91.8505192, -91.8505040, -91.8504868, -9...
$ lat          <dbl> 40.376043052, 40.376043152, 40.376042752, 40.376042552...
$ variety      <chr> "P1498", "P1498", "P1498", "P1498", "P1498", "P1498", ...
$ seeding_rate <dbl> 47.6142, 47.6412, 47.6412, 47.6412, 34.6421, 31.2031, ...
$ seed_spacing <dbl> 4.3913, 4.3888, 4.3888, 4.3888, 6.0357, 6.7009, 7.0360...
$ speed        <dbl> 1.90, 2.11, 2.52, 2.94, 3.37, 3.88, 4.22, 4.26, 4.35, ...


Let's check if the gps coordinates overlap between the planter.csv and harvest.csv.

In [5]:
cat(nrow(semi_join(harvester, planter, by=c("long","lat"))), fill=T)

0


It is clear that none of the points overlap. Now let us visualize the gps coordinates to get more insights.

## Data Consistency

Before we visualize the data, let's first check the data consistency.

1. Are the range of gps coordinates from planter.csv and harvest.csv in sync? i.e. within a small tolerance.
2. Are the gps coordinates confined within a reasonable bounding-box? i.e. must be a finite area
3. Do the points overlay over a resonable area? i.e. must be on land
    
Let's find the range of the gps coordinates for the **bounding box**.

In [6]:
b_long <- c(min(min(planter$long),min(harvester$long)), max(max(planter$long),max(harvester$long)))
b_lat <- c(min(min(planter$lat),min(harvester$lat)), max(max(planter$lat),max(harvester$lat)))
cat("long-min: ",b_long[1], ",", "long-max: ",b_long[2], fill=T)
cat("lat-min: ",b_lat[1], ",", "lat-max: ",b_lat[2], fill=T)

long-min:  -91.850657942 , long-max:  -91.845947571
lat-min:  40.375880452 , lat-max:  40.379421396


Let's find the **field dimensions**.

In [7]:
length <- round(distm(c(b_long[1], b_lat[1]), c(b_long[1], b_lat[2]), fun = distHaversine),1)
breadth <- round(distm(c(b_long[1], b_lat[1]), c(b_long[2], b_lat[1]), fun = distHaversine),1)
cat("y: ", length,"m" , fill=T)
cat("x: ", breadth,"m", fill=T)

y:  394.2 m
x:  399.5 m


This field is approximately 400m x 400m field. i.e. ~40 acres. This is reasonable.

Let's find the **lat-long offset per metre**.

In [8]:
lat.pm <- (b_lat[2]-b_lat[1])/length
long.pm <- (b_long[2]-b_long[1])/breadth
cat("Per metre latitude offset: ", lat.pm, fill=T)
cat("Per metre longitude offset: ", long.pm, fill=T)

Per metre latitude offset:  8.9826090416e-06
Per metre longitude offset:  1.1790665833e-05


Now, let's find the **field center**. This is to be used for map center.

In [9]:
hc <- c((max(harvester$long)+min(harvester$long))/2, (max(harvester$lat)+min(harvester$lat))/2)
pc <- c((max(planter$long)+min(planter$long))/2, (max(planter$lat)+min(planter$lat))/2)
cat("harvest-long: ",hc[1], ",", "harvest-lat: ",hc[2], fill=T)
cat("planter-long: ",pc[1], ",", "planter-lat: ",pc[2], fill=T)

harvest-long:  -91.848302757 , harvest-lat:  40.377655013
planter-long:  -91.848297 , planter-lat:  40.377635252


The lat-long coordinates for center of the field for harvester and planter are in sync. The longitude is -ve and the latitude is +ve. So this must be in North-West. Let's confirm this with the map.

In [10]:
options(repr.plot.width=8, repr.plot.height=8)
map <- c(lon = -91.848302, lat = 40.377655)
map <- get_map(location = map, zoom = 12)
ggsave('images/fieldmap.jpg', plot = ggmap(map), width = 8, height = 8)

Map from URL : http://maps.googleapis.com/maps/api/staticmap?center=40.377655,-91.848302&zoom=12&size=640x640&scale=2&maptype=terrain&language=en-EN&sensor=false


![Fieldmap](images/fieldmap.jpg)

The map is centered somewhere over Wyaconda, which is in Missouri, USA.

## Exploratory Analysis

Now that the data consistency is verified, let's plot the gps coordinates.

In [11]:
g1 <- ggplot() + geom_point(aes(long, lat, alpha=0.01), data = planter) +
                 geom_point(aes(long, lat, alpha=0.01, colour = "red"), data = harvester)
ggsave('images/planter-harvester.jpg', plot = g1, width = 32, height = 32)

![Planter-Harvester](images/planter-harvester.jpg)

### Observations

1. More harvester gps-cordinates than planter. Besides the harvester machine has run exactly on either side of the planter machine path. The image below on the left is that of a 32 row corn planter and the image on the right is a multi-row corn harvester.
![Planter-Harvester](images/planter-harvester-machine.jpg)
2. **Data Provenance** - Where is/are the gps devices mounted? How are the variables such as yield, speed, seeding rate & seed spacing recorded?
3. 66 vertical lines for harvester vs 33 vertical lines for planter. 8 horizontal lines for harvester vs 4 horizontal lines for planter. So harvester is ~2X+ that of planter based on the lines, but with higher density of data collection i.e. 16626 vs 6314 data points.
4. In this case a similar multi-row equipment could have been used - for example a 16 row planter and a 8 row harvester. That explains why the `harvester` file has 2X+ gps coordinates compared to `planter` file.
5. As you can see the lines are not exactly vertical or horizontal. So there is no direct option to group the lines.

## Algorithm

The brute force approach would be to take each lat-long coordinate from harvest.csv and find the nearest lat-long coordinate in the planter.csv. This would be computationaly intensive as 16626 x 6314 distance computations need to be carried out.

The heuristic approach would be to categorize the gps coordinates into smaller bins to restrict the search space. i.e. divide the field into grids similar to a chessboard. In the **uniform grid** approach the field is divided into equal grids by equal lat and long offsets.

### Gridline design
Now let's design the grid layout and then overlay it over the gps coordinates. Let's go with a 20m x 20m grid. i.e. ~10 cents or 0.1 acre. To cover the entire field (40 acres) in this case we would need ~400 grids.

In [12]:
# vertical gridlines
nx <- ceiling((breadth+(long.pm * 5))/10)
x <- seq(from=b_long[1]-(long.pm * 5), length.out=nx+2, by=(long.pm * 10))
cat("Vertical gridlines: ",length(x), fill=T)

# horizontal gridlines 
ny <- ceiling((length+(lat.pm * 5))/10)
y <- seq(from=b_lat[1]-(lat.pm * 5), length.out=ny+2, by=(lat.pm * 10))
cat("Horizontal gridlines: ",length(y), fill=T)

Vertical gridlines:  42
Horizontal gridlines:  42


### Overlay gridlines

In [13]:
for(i in 1:length(x)){
    g1 <- g1+geom_vline(xintercept = x[i])+geom_hline(yintercept = y[i])
}
ggsave('images/planter-harvester-grids.jpg', plot = g1, width = 32, height = 32)

![Planter-Harvester-Grids](images/planter-harvester-grids.jpg)

### Assign grid memberships

Now that we have visualized the grid layout, let's assign the geo-coodinates to each unique grid-box. In this case there are 40x40 grids i.e. 1600 grids.

In this case I use the `foverlaps()` function from `data.table` package to do this. The steps are as follows,

1. Create a table with long ranges (x-axis) mapped to  grid sequence i.e. 1 to 40
2. Create a table with lat ranges (y-axis) mapped to  grid sequence i.e. 1 to 40
3. Create duplicate cols in planter & harvesterfor lat & long coordinates
4. Setkey in the above two tables and planter & harvester (for long coordinates) 
5. `foverlaps()` to left join horizontal/column grid membership
6. Repeat the steps 4,5 for lat coordinates

In [14]:
# create long grid ranges
dfx <- tbl_df(x)
dfx$value1 <- lead(dfx$value)-1e-10
dfx <- dfx %>% na.omit()
dfx$groupX <- 1:nrow(dfx)
dfx <- data.table(dfx)

# create lat grid ranges
dfy <- tbl_df(y)
dfy$value1 <- lead(dfy$value)-1e-10
dfy <- dfy %>% na.omit()
dfy$groupY <- 1:nrow(dfy)
dfy <- data.table(dfy)

glimpse(dfx)
glimpse(dfy)

Observations: 41
Variables: 3
$ value  <dbl> -91.850716895, -91.850598989, -91.850481082, -91.850363175, ...
$ value1 <dbl> -91.850598989, -91.850481082, -91.850363176, -91.850245269, ...
$ groupX <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...
Observations: 41
Variables: 3
$ value  <dbl> 40.375835538, 40.375925365, 40.376015191, 40.376105017, 40.3...
$ value1 <dbl> 40.375925364, 40.376015191, 40.376105017, 40.376194843, 40.3...
$ groupY <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...


The values on the left col must be less than the right in the range. Let's verify

In [15]:
cat(sum(dfx$value<=dfx$value1), fill=T)
cat(sum(dfy$value<=dfy$value1), fill=T)

41
41


Now that the grid ranges are verified, let's asign unique group/grid number

In [16]:
group <- expand.grid(groupX=c(1:nrow(dfx)),groupY=c(1:nrow(dfy)))
group$value <- paste0(group$groupY,"-",group$groupX)
group$group <- 1:nrow(group)
group <- group %>% select(value, group)
glimpse(group)

Observations: 1,681
Variables: 2
$ value <chr> "1-1", "1-2", "1-3", "1-4", "1-5", "1-6", "1-7", "1-8", "1-9"...
$ group <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18...


#### Map 'planter' gps coordinates to unique grids

In [17]:
# create dummy lat-long ranges
planter <- planter %>%
                mutate(long1 = round(long,5),
                       long2 = round(long,5),
                       lat1 = round(lat,6),
                       lat2 = round(lat,6))
planter <- data.table(planter)

#setkey
setkey(planter, long1, long2)
setkey(dfx, value, value1)

#vertical grids
planter_grid <- foverlaps(planter, dfx, nomatch = NA) #left_join

#setkey
setkey(planter_grid, lat1, lat2)
setkey(dfy, value, value1)

#horizontal grids
planter_grid <- foverlaps(planter_grid, dfy, nomatch = NA) #left_join

planter_grid <- planter_grid %>%
    select_("long","lat","variety","seeding_rate","seed_spacing","speed","groupX","groupY") %>%
    mutate(value = paste0(groupY,"-",groupX)) %>% select(-groupX, -groupY) %>%
    left_join(group, by="value") %>% arrange(long,lat)

glimpse(planter_grid)

Observations: 6,314
Variables: 8
$ long         <dbl> -91.8506140, -91.8506124, -91.8506117, -91.8506114, -9...
$ lat          <dbl> 40.375880452, 40.375968352, 40.376001652, 40.376025952...
$ variety      <chr> "DKC63-33RIB", "DKC63-33RIB", "DKC63-33RIB", "DKC63-33...
$ seeding_rate <dbl> 0.0000, 0.0000, 0.0000, 31.7371, 32.8151, 33.4241, 34....
$ seed_spacing <dbl> 0.0000, 0.0000, 0.0000, 6.5881, 6.3717, 6.2556, 6.1480...
$ speed        <dbl> 0.02, 3.75, 3.76, 3.77, 3.76, 3.75, 3.74, 3.72, 3.42, ...
$ value        <chr> "1-1", "2-1", "2-1", "3-1", "3-1", "3-1", "3-1", "3-1"...
$ group        <int> 1, 42, 42, 83, 83, 83, 83, 83, 124, 124, 124, 124, 124...


#### Map 'harvest' gps coordinates to unique grids

In [18]:
# create dummy lat-long ranges
harvester <- harvester %>%
                mutate(long1 = round(long,5),
                       long2 = round(long,5),
                       lat1 = round(lat,6),
                       lat2 = round(lat,6))
harvester <- data.table(harvester)

#setkey
setkey(harvester, long1, long2)

#left_join
harvest_grid <- foverlaps(harvester, dfx, nomatch = NA) #left_join

#setkey
setkey(harvest_grid, lat1, lat2)
#left_join
harvest_grid <- foverlaps(harvest_grid, dfy, nomatch = NA) #left_join

harvest_grid <- harvest_grid %>% 
    select_("long","lat","yield","groupX","groupY") %>%
    mutate(value = paste0(groupY,"-",groupX)) %>% select(-groupX, -groupY) %>%
    left_join(group, by="value") %>% arrange(long,lat)

glimpse(harvest_grid)

Observations: 16,626
Variables: 5
$ long  <dbl> -91.850657942, -91.850656829, -91.850656369, -91.850655826, -...
$ lat   <dbl> 40.375893366, 40.375903539, 40.375914728, 40.375926576, 40.37...
$ yield <dbl> 216.92110846, 218.21178574, 241.78874780, 227.94857084, 226.6...
$ value <chr> "1-1", "1-1", "1-1", "2-1", "2-1", "2-1", "2-1", "2-1", "2-1"...
$ group <int> 1, 1, 1, 42, 42, 42, 42, 42, 42, 42, 83, 83, 83, 83, 83, 83, ...


Let's verify the data consistency between the original file and grid assigned file.

In [19]:
cat(setequal(select(planter,-long1,-long2,-lat1,-lat2),
         select(planter_grid,-value,-group)), fill=T)
cat(setequal(select(harvester,-long1,-long2,-lat1,-lat2),
         select(harvest_grid,-value,-group)), fill=T)

TRUE
TRUE


### Mapping planter to harvester

In [20]:
# Create new variable to store data from planter
harvest_grid <- harvest_grid %>%
    mutate(variety = NA,
           seeding_rate = NA,
           seed_spacing = NA,
           speed = NA,
           long1 = NA,
           lat1 = NA,
           dist = NA)
glimpse(harvest_grid)

Observations: 16,626
Variables: 12
$ long         <dbl> -91.850657942, -91.850656829, -91.850656369, -91.85065...
$ lat          <dbl> 40.375893366, 40.375903539, 40.375914728, 40.375926576...
$ yield        <dbl> 216.92110846, 218.21178574, 241.78874780, 227.94857084...
$ value        <chr> "1-1", "1-1", "1-1", "2-1", "2-1", "2-1", "2-1", "2-1"...
$ group        <int> 1, 1, 1, 42, 42, 42, 42, 42, 42, 42, 83, 83, 83, 83, 8...
$ variety      <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ seeding_rate <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ seed_spacing <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ speed        <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ long1        <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ lat1         <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ dist         <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...


The nearest neighbour grid search search is done horizontally and vertically. For example if the selected grid is 3-3, then the nearest neighbour grids are as below.

In [21]:
n <- unlist(strsplit('3-3', '-'))
row <- as.integer(n[1])
col <- as.integer(n[2])   
neighbour <- c(paste0(row,"-",col-1),
               paste0(row-1,"-",col),
               paste0(row,"-",col),
               paste0(row+1,"-",col),
               paste0(row,"-",col+1))  
cat(neighbour, fill=T)

3-2 2-3 3-3 4-3 3-4


Putting it all together. The code takes ~4 min to run. Please be patient.

In [22]:
timer <- Sys.time()
#pb <- txtProgressBar(max=nrow(harvest_grid), style = 3)
for(i in 1:nrow(harvest_grid)){
    harvest.gps <- harvest_grid[i,]
    
    n <- unlist(strsplit(harvest.gps$value, '-'))
    row <- as.integer(n[1])
    col <- as.integer(n[2])        
    # filter the rows from planter - nearest neighbour search      
    neighbour <- c(paste0(row,"-",col-1),
                   paste0(row-1,"-",col),
                   paste0(row,"-",col),
                   paste0(row+1,"-",col),
                   paste0(row,"-",col+1))       
    planter.gps <- planter_grid %>% filter(value %in% neighbour)
    
    if(nrow(planter.gps)>0){
        # store gps coordinates for vector based operation
        planter.gps$long1 <- harvest.gps$long
        planter.gps$lat1 <- harvest.gps$lat
        
        # find the distance between gps coordinates
        planter.gps$dist <- distm(as.matrix(planter.gps[,c('long','lat')]), as.matrix(planter.gps[,c('long1','lat1')]), fun = distHaversine)[,1]
        planter.gps <- planter.gps %>% filter(dist == min(dist)) %>% select(-long1, -lat1, -group)
        
        # map the data back to harvest
        harvest_grid[i, "variety"] <- planter.gps[1,"variety"]
        harvest_grid[i, "seeding_rate"] <- planter.gps[1,"seeding_rate"]
        harvest_grid[i, "seed_spacing"] <- planter.gps[1,"seed_spacing"]
        harvest_grid[i, "speed"] <- planter.gps[1,"speed"]
        harvest_grid[i, "long1"] <- planter.gps[1,"long"]
        harvest_grid[i, "lat1"] <- planter.gps[1,"lat"]
        harvest_grid[i, "dist"] <- planter.gps[1,"dist"]
    }
    #setTxtProgressBar(pb, i)
}
cat("***Process completed in",round(difftime(Sys.time(), timer, tz = "", units = "mins"),2),"min!***", fill=T)

***Process completed in 3.34 min!***


In [24]:
glimpse(harvest_grid)

Observations: 16,626
Variables: 12
$ long         <dbl> -91.850657942, -91.850656829, -91.850656369, -91.85065...
$ lat          <dbl> 40.375893366, 40.375903539, 40.375914728, 40.375926576...
$ yield        <dbl> 216.92110846, 218.21178574, 241.78874780, 227.94857084...
$ value        <chr> "1-1", "1-1", "1-1", "2-1", "2-1", "2-1", "2-1", "2-1"...
$ group        <int> 1, 1, 1, 42, 42, 42, 42, 42, 42, 42, 83, 83, 83, 83, 8...
$ variety      <chr> "DKC63-33RIB", "DKC63-33RIB", "DKC63-33RIB", "DKC63-33...
$ seeding_rate <dbl> 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000...
$ seed_spacing <dbl> 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000...
$ speed        <dbl> 0.02, 0.02, 0.02, 3.75, 3.75, 3.75, 3.75, 3.75, 3.76, ...
$ long1        <dbl> -91.8506140, -91.8506140, -91.8506140, -91.8506124, -9...
$ lat1         <dbl> 40.375880452, 40.375880452, 40.375880452, 40.375968352...
$ dist         <dbl> 3.9942087025, 4.4494600649, 5.2411452085, 5.9320448613...


## Efficiency Metric

Besides the time taken to complete, the distance between planter and harvester data points is the metric which decides if the points are tightly clustered. 

Let's visualize the histogram of distance between planter and harvester.

In [25]:
plot_hist <- harvest_grid %>% ggplot(aes(x=dist)) + geom_histogram()
ggsave('images/planter-harvester-hist.jpg', plot = plot_hist, width = 8, height = 4)

`stat_bin()` using `bins = 30`. Pick better value with `binwidth`.


![Planter-Harvester-Histogram](images/planter-harvester-hist.jpg)

The distance is mostly 3 metres i.e. 10 feet, which is reasonable given the fact that the precision machines are multi row.

### Save the file

In [26]:
write.csv(harvest_grid, file = "planter-harvester.csv", row.names=FALSE)

### Note

1. The ‘haversine’ formula is used to calculate distance between 2 gps coordinates. Other formulae like Cosine, VincentySphere, VincentyEllipsoid can be used.
2. The operation is sequential and takes ~4 min. The entire process could be executed in parallel _(using `foreach()`)_ to improve efficiency, as all computations are independent.
3. Distance between longitude and latitude is constant over a smaller area. i.e. the field is flat. In reality this isn't the case. The distance between longitudes is 0km at poles and ~133km at the equator. The distance between latitudes is ~111km.

## Statistical Analysis

Let's group the data by the corn variety.

In [27]:
harvest_grid %>% group_by(variety) %>% summarise(count = n(), sum = sum(yield), sd = sd(yield))

,variety,count,sum,sd
1,DKC63-33RIB,14372,3332944.0397097,25.0110720021795
2,P1498,2254,510186.572986916,34.6597138288796


There are just two variety of corn. Looks like `DKC63-33RIB` is the main one, while `P1498` is a trail for yield.

Besides from my understanding of corn farming, a genetically modified variety yields ~200 bushels/acre. But each data point in the `harvester_data.csv` has yield ~200 with the unit bushels/acre as mentioned. This isn't possible as each gps point isn't an acre.

Now let's plot a density plot for yield for both the corn variety.

In [28]:
plot_den <- harvest_grid %>% ggplot(aes(x=yield, fill=variety)) +
    geom_histogram(aes(y=..density..), binwidth=.5, alpha=.5, position="identity")    
ggsave('images/corn-variety.jpg', plot = plot_den, width = 8, height = 4)

![Corn-Variety](images/corn-variety.jpg)

Looks like the `P1498` has more instances where the yield is below 200 units.

The next logical question would be - is `P1498` variety significantly different from `DKC63-33RIB` variety in terms of yield? Let's check the mean first.

In [29]:
harvest_grid %>% group_by(variety) %>% summarise(count = n(), mean = mean(yield), sd = sd(yield))

,variety,count,mean,sd
1,DKC63-33RIB,14372,231.905374318793,25.0110720021795
2,P1498,2254,226.347192984435,34.6597138288796


A **two tailed t-test** is required to determine if the means are statistically different.

Let's frame the **hypothesis**:

$ H_0: \mu_{DKC63-33RIB} - \mu_{P1498} = 0$

$ H_1: \mu_{DKC63-33RIB} - \mu_{P1498} \neq 0$

In [31]:
h <- t.test(filter(harvest_grid, variety=='DKC63-33RIB') %>% .$yield,
          filter(harvest_grid, variety=='P1498') %>% .$yield,
          paired = F, var.equal = FALSE)
cat("Confidence interval (CI):", h$conf.int, fill=T)
cat("P-value:", h$p.value, fill=T)

Confidence interval (CI): 4.0693594537 7.047003215
P-value: 3.2667180289e-13


The p-value is less than 0.05 (assuming a 95% CI) and 0 isn't in the CI. So we reject the $H_0$. In this case `DKC63-33RIB` variety has better yield compared to the `P1498` variety by 4-7 units per acre.

Next, let's check the influence of the variables - seeding rate, seed spacing and speed on yield for the major variety `DKC63-33RIB`.

First, let's examine each of these variables.

In [32]:
DKC63_33RIB <- harvest_grid %>% filter(variety == 'DKC63-33RIB')
p1 <- DKC63_33RIB %>% ggplot(aes(x=seeding_rate)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
p2 <- DKC63_33RIB %>% ggplot(aes(x=seed_spacing)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
p3 <- DKC63_33RIB %>% ggplot(aes(x=speed)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
plot_p1 <- plot_grid(p1,p2,p3, nrow = 1, ncol = 3)
ggsave('images/DKC63_33RIB.jpg', plot = plot_p1, width = 8, height = 3)

![Corn-Variety-DKC63_33RIB](images/DKC63_33RIB.jpg)

In [33]:
P1498 <- harvest_grid %>% filter(variety == 'P1498')
p1 <- P1498 %>% ggplot(aes(x=seeding_rate)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
p2 <- P1498 %>% ggplot(aes(x=seed_spacing)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
p3 <- P1498 %>% ggplot(aes(x=speed)) + geom_histogram(binwidth=.5, alpha=.5, position="identity")
plot_p2 <- plot_grid(p1,p2,p3, nrow = 1, ncol = 3)
ggsave('images/P1498.jpg', plot = plot_p2, width = 8, height = 3)

![Corn-Variety-P1498](images/P1498.jpg)

Let's check for the correlations.

In [34]:
cor(DKC63_33RIB %>% select(yield, seeding_rate, seed_spacing, speed))

,yield,seeding_rate,seed_spacing,speed
yield,1.000000000000,0.025440208633,-0.028035536397,0.055932730337
seeding_rate,0.025440208633,1.000000000000,-0.110126202364,0.315066766633
seed_spacing,-0.028035536397,-0.110126202364,1.000000000000,0.011788013335
speed,0.055932730337,0.315066766633,0.011788013335,1.000000000000


In [35]:
cor(P1498 %>% select(yield, seeding_rate, seed_spacing, speed))

,yield,seeding_rate,seed_spacing,speed
yield,1.000000000000,0.101515181511,-0.089742169382,-0.022399522796
seeding_rate,0.10151518151,1.00000000000,-0.97720824192,0.10073656335
seed_spacing,-0.089742169382,-0.977208241916,1.000000000000,-0.129996584027
speed,-0.022399522796,0.100736563347,-0.129996584027,1.000000000000


Only seeding_rate and seed_spacing are strongly negatively correlated. None of the variables are correlated with yield.

From my understanding on corn farming _from the documentary [King Corn (2007)](http://www.imdb.com/title/tt1112115)_,

- The kernels(seeds) are sown in April/May with fertilizer.
- Fertilizer: ~150 pound anhydrous ammonia per acre. This depends on the type of soil.
- ~31000 kernels planted per acre.
- Herbicide sprayed in June.
- Harvested in October/November.
    
So essentially there are several factors in play that influence yield.

## SessionInfo

In [36]:
sessionInfo()

R version 3.3.1 (2016-06-21)
Platform: x86_64-apple-darwin11.0.0 (64-bit)
Running under: OS X 10.10.5 (Yosemite)

locale:
[1] C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] cowplot_0.7.0     data.table_1.10.4 dplyr_0.5.0       geosphere_1.5-5  
[5] sp_1.2-4          ggmap_2.6.1       ggplot2_2.2.1    

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.9       plyr_1.8.4        tools_3.3.1       digest_0.6.9     
 [5] uuid_0.1-2        jsonlite_1.2      evaluate_0.9      memoise_1.0.0    
 [9] tibble_1.2        gtable_0.2.0      lattice_0.20-34   png_0.1-7        
[13] IRdisplay_0.4.3   DBI_0.5-1         mapproj_1.2-4     IRkernel_0.7     
[17] proto_1.0.0       repr_0.7          stringr_1.1.0     RgoogleMaps_1.4.1
[21] maps_3.1.1        grid_3.3.1        R6_2.1.2          jpeg_0.1-8       
[25] pbdZMQ_0.2-3      readr_1.0.0       reshape2_1.4.2    magrittr_1.5     
[29] scales_0.4.1      assertt